In [4]:
# ==========================================================
# Seattle Commuting Flows (LODES8, WA 2022)
# - Block, Block Group, and Tract levels
# - GEOIDs derived directly from 15-digit block codes (2020-compatible)
# - Destinations filtered to polygons that INTERSECT Seattle
# - Centroids computed in projected CRS; diagnostics printed
# - Exports: block / blockgroup / tract CSVs + combined CSV
# ==========================================================

import os, io, zipfile, requests
from pathlib import Path
import pandas as pd
import geopandas as gpd

# -----------------------------
# Config & helpers
# -----------------------------
DATA = Path("data"); DATA.mkdir(exist_ok=True)

def fetch(url: str, out_path: Path):
    if out_path.exists():
        print(f"✅ Exists: {out_path.name}")
        return out_path
    print(f"⬇️  Downloading: {url}")
    r = requests.get(url, timeout=180)
    r.raise_for_status()
    out_path.write_bytes(r.content)
    print(f"✅ Saved: {out_path.name}")
    return out_path

def fetch_unzip(url: str, out_dir: Path) -> Path:
    out_dir.mkdir(exist_ok=True, parents=True)
    shp = list(out_dir.glob("*.shp"))
    if shp:
        print(f"✅ Exists: {out_dir.name}")
        return shp[0]
    print(f"⬇️  Downloading & unzipping: {url}")
    r = requests.get(url, timeout=240)
    r.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(r.content)) as z:
        z.extractall(out_dir)
    shp = list(out_dir.glob("*.shp"))
    print(f"✅ Unzipped: {out_dir} ({shp[0].name})")
    return shp[0]

def geounion(gdf: gpd.GeoDataFrame):
    # union_all for newer versions; fallback to unary_union
    return gdf.union_all() if hasattr(gdf, "union_all") else gdf.unary_union

def centroid_lonlat(gdf: gpd.GeoDataFrame, proj_epsg=3857):
    g_proj = gdf.to_crs(proj_epsg)
    cents = g_proj.geometry.centroid
    cents_ll = gpd.GeoSeries(cents, crs=g_proj.crs).to_crs(4326)
    return pd.DataFrame({"lon": cents_ll.x.values, "lat": cents_ll.y.values})

def log(msg): print(f"▶ {msg}")

# -----------------------------
# 1) Download inputs
# -----------------------------
od_fp  = fetch("https://lehd.ces.census.gov/data/lodes/LODES8/wa/od/wa_od_main_JT00_2022.csv.gz",
               DATA / "wa_od_main_JT00_2022.csv.gz")
xw_fp  = fetch("https://lehd.ces.census.gov/data/lodes/LODES8/wa/wa_xwalk.csv.gz",
               DATA / "wa_xwalk.csv.gz")
tracts_shp  = fetch_unzip("https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_53_tract.zip",
                          DATA / "tl_2021_53_tract")
bg_shp      = fetch_unzip("https://www2.census.gov/geo/tiger/TIGER2021/BG/tl_2021_53_bg.zip",
                          DATA / "tl_2021_53_bg")
places_shp  = fetch_unzip("https://www2.census.gov/geo/tiger/TIGER2021/PLACE/tl_2021_53_place.zip",
                          DATA / "tl_2021_53_place")

# -----------------------------
# 2) Load OD and derive GEOIDs from block codes (2020-compatible)
# -----------------------------
od = pd.read_csv(od_fp, dtype={"h_geocode":"string","w_geocode":"string"})
od["S000"] = od["S000"].astype("int64")
log(f"OD rows: {len(od):,}")

# Ensure 15 digits; derive tract (11) and block group (12) from block IDs
od["h_geocode"] = od["h_geocode"].str.zfill(15)
od["w_geocode"] = od["w_geocode"].str.zfill(15)
od["tract_home"] = od["h_geocode"].str[:11]
od["tract_work"] = od["w_geocode"].str[:11]
od["bg_home"]    = od["h_geocode"].str[:12]
od["bg_work"]    = od["w_geocode"].str[:12]
log(f"Unique work tracts: {od['tract_work'].nunique():,} | work BGs: {od['bg_work'].nunique():,}")

# For block-level coordinates we can use crosswalk block coords (lat/lon)
xw = pd.read_csv(xw_fp, dtype={"tabblk2020":"string"})
xw["tabblk2020"] = xw["tabblk2020"].str.zfill(15)
blk_coords = xw.rename(columns={"tabblk2020":"geocode",
                                "blklondd":"lon",
                                "blklatdd":"lat"})[["geocode","lon","lat"]]

# -----------------------------
# 3) Load TIGER shapes (WGS84) + Seattle polygon
# -----------------------------
tracts = gpd.read_file(tracts_shp).to_crs(4326)
tracts["GEOID"] = tracts["GEOID"].astype(str).str.zfill(11)
bg = gpd.read_file(bg_shp).to_crs(4326)
bg["GEOID"] = bg["GEOID"].astype(str).str.zfill(12)

places = gpd.read_file(places_shp).to_crs(4326)
seattle = places.query("NAME == 'Seattle'").copy()
assert len(seattle) == 1, "Seattle boundary not found or multiple matches."
SEA = geounion(seattle)
log("Seattle polygon loaded")

# Precompute accurate centroids for BG & tracts (projected → back to lon/lat)
bg_cent = centroid_lonlat(bg)
bg = pd.concat([bg.reset_index(drop=True), bg_cent], axis=1)     # adds lon/lat
tracts_cent = centroid_lonlat(tracts)
tracts = pd.concat([tracts.reset_index(drop=True), tracts_cent], axis=1)

# -----------------------------
# 4) Build ID sets of destinations that touch Seattle (intersects)
# -----------------------------
# Do spatial tests in a metric CRS for robustness
bg_m = bg.to_crs(3857); tracts_m = tracts.to_crs(3857); seattle_m = seattle.to_crs(3857)
SEA_M = geounion(seattle_m)

bg_ids_in_sea  = set(bg_m[bg_m.intersects(SEA_M)].index)
tr_ids_in_sea  = set(tracts_m[tracts_m.intersects(SEA_M)].index)
bg_geoids_in_sea = set(bg.iloc[list(bg_ids_in_sea)]["GEOID"]) if bg_ids_in_sea else set()
tr_geoids_in_sea = set(tracts.iloc[list(tr_ids_in_sea)]["GEOID"]) if tr_ids_in_sea else set()
log(f"BGs intersecting Seattle: {len(bg_geoids_in_sea):,}")
log(f"Tracts intersecting Seattle: {len(tr_geoids_in_sea):,}")

# -----------------------------
# 5) BLOCK → BLOCK flows (coords from crosswalk; polygon filter)
# -----------------------------
flows_block = od[["h_geocode","w_geocode","S000"]].rename(columns={"S000":"jobs"})
flows_block = flows_block.merge(blk_coords, left_on="h_geocode", right_on="geocode", how="left") \
                         .rename(columns={"lon":"home_lon","lat":"home_lat"}).drop(columns=["geocode"])
flows_block = flows_block.merge(blk_coords, left_on="w_geocode", right_on="geocode", how="left") \
                         .rename(columns={"lon":"work_lon","lat":"work_lat"}).drop(columns=["geocode"])

flows_block_gdf = gpd.GeoDataFrame(
    flows_block,
    geometry=gpd.points_from_xy(flows_block["work_lon"], flows_block["work_lat"]),
    crs=4326
).to_crs(3857)

flows_block_sea = flows_block_gdf[flows_block_gdf.intersects(SEA_M)] \
                      .dropna(subset=["home_lon","home_lat","work_lon","work_lat"]).copy()
flows_block_sea["geo_level"] = "block"
log(f"Block-level flows (Seattle dest): {len(flows_block_sea):,}")

# -----------------------------
# 6) BLOCK GROUP → BLOCK GROUP flows (ID-based dest filter)
# -----------------------------
# Filter by destination BG GEOIDs that touch Seattle, then aggregate
od_bg = od[od["bg_work"].isin(bg_geoids_in_sea)].copy()
log(f"OD rows with BG-work in Seattle: {len(od_bg):,}")

flows_bg = (od_bg.groupby(["bg_home","bg_work"], as_index=False)["S000"]
                 .sum().rename(columns={"S000":"jobs"}))
log(f"BG→BG pairs: {len(flows_bg):,}")

# Attach BG centroids
bg_pts = bg[["GEOID","lon","lat"]].copy()
flows_bg = flows_bg.merge(bg_pts, left_on="bg_home", right_on="GEOID", how="left") \
                   .rename(columns={"lon":"home_lon","lat":"home_lat"}).drop(columns=["GEOID"])
flows_bg = flows_bg.merge(bg_pts, left_on="bg_work", right_on="GEOID", how="left") \
                   .rename(columns={"lon":"work_lon","lat":"work_lat"}).drop(columns=["GEOID"])
flows_bg = flows_bg.dropna(subset=["home_lon","home_lat","work_lon","work_lat"]).copy()
flows_bg["geo_level"] = "blockgroup"
log(f"Block-group flows (final): {len(flows_bg):,}")

# -----------------------------
# 7) TRACT → TRACT flows (ID-based dest filter)
# -----------------------------
od_tr = od[od["tract_work"].isin(tr_geoids_in_sea)].copy()
log(f"OD rows with tract-work in Seattle: {len(od_tr):,}")

flows_tr = (od_tr.groupby(["tract_home","tract_work"], as_index=False)["S000"]
                 .sum().rename(columns={"S000":"jobs"}))
log(f"Tract→Tract pairs: {len(flows_tr):,}")

# Attach tract centroids
tr_pts = tracts[["GEOID","lon","lat"]].copy()
flows_tr = flows_tr.merge(tr_pts, left_on="tract_home", right_on="GEOID", how="left") \
                   .rename(columns={"lon":"home_lon","lat":"home_lat"}).drop(columns=["GEOID"])
flows_tr = flows_tr.merge(tr_pts, left_on="tract_work", right_on="GEOID", how="left") \
                   .rename(columns={"lon":"work_lon","lat":"work_lat"}).drop(columns=["GEOID"])
flows_tr = flows_tr.dropna(subset=["home_lon","home_lat","work_lon","work_lat"]).copy()
flows_tr["geo_level"] = "tract"
log(f"Tract flows (final): {len(flows_tr):,}")

# -----------------------------
# 8) Save outputs (and combined)
# -----------------------------
cols = ["home_lon","home_lat","work_lon","work_lat","jobs","geo_level"]

flows_block_sea[cols].to_csv(DATA/"seattle_flows_block_2022.csv", index=False)
flows_bg[cols].to_csv(DATA/"seattle_flows_blockgroup_2022.csv", index=False)
flows_tr[cols].to_csv(DATA/"seattle_flows_tract_2022.csv", index=False)

flows_all = pd.concat([flows_block_sea[cols], flows_bg[cols], flows_tr[cols]], ignore_index=True)
flows_all.to_csv(DATA/"seattle_flows_multilevel_2022.csv", index=False)

print("✅ Saved:")
print(" - data/seattle_flows_block_2022.csv")
print(" - data/seattle_flows_blockgroup_2022.csv")
print(" - data/seattle_flows_tract_2022.csv")
print(" - data/seattle_flows_multilevel_2022.csv  (all levels in one file)")

# -----------------------------
# 9) (Optional) Thin tiny flows for Kepler performance
# -----------------------------
# flows_all_min20 = flows_all[flows_all["jobs"] >= 20].copy()
# flows_all_min20.to_csv(DATA/"seattle_flows_multilevel_2022_min20.csv", index=False)
# print("✅ Also saved: data/seattle_flows_multilevel_2022_min20.csv (jobs >= 20)")


✅ Exists: wa_od_main_JT00_2022.csv.gz
✅ Exists: wa_xwalk.csv.gz
✅ Exists: tl_2021_53_tract
✅ Exists: tl_2021_53_bg
✅ Exists: tl_2021_53_place
▶ OD rows: 2,905,296
▶ Unique work tracts: 1,771 | work BGs: 5,292


C:\Users\lette\AppData\Local\Temp\ipykernel_618512\2803506025.py:89: DtypeWarning: Columns (31,35) have mixed types. Specify dtype option on import or set low_memory=False.
  xw = pd.read_csv(xw_fp, dtype={"tabblk2020":"string"})


▶ Seattle polygon loaded
▶ BGs intersecting Seattle: 571
▶ Tracts intersecting Seattle: 199
▶ Block-level flows (Seattle dest): 544,781
▶ OD rows with BG-work in Seattle: 563,744
▶ BG→BG pairs: 281,474
▶ Block-group flows (final): 281,474
▶ OD rows with tract-work in Seattle: 578,207
▶ Tract→Tract pairs: 122,950
▶ Tract flows (final): 122,950
✅ Saved:
 - data/seattle_flows_block_2022.csv
 - data/seattle_flows_blockgroup_2022.csv
 - data/seattle_flows_tract_2022.csv
 - data/seattle_flows_multilevel_2022.csv  (all levels in one file)
